In [4]:
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe
%use kandy

import ro.jf.funds.client.notebook.*
import ro.jf.funds.reporting.api.model.*

val username = "Johann-27.4"

val monthlyReportFrom = YearMonthTO(2019, 1)
val monthlyReportTo = YearMonthTO(2021, 3)
val monthlyReportForecastUntil = YearMonthTO(2021, 9)

val yearlyReportFrom = 2019
val yearlyReportTo = 2020
val yearlyReportForecastUntil = 2022

val fundName = "Expenses"
val reportViewName = "Expenses report"

val REPORT_DATA_CONFIGURATION_YAML_FILE = "../../data/provision/expenses-report-data-configuration.yaml"

val client = FundsClient()

In [5]:
val user = client.ensureUserExists(username)
user

UserTO(id=9fab0ce6-a04f-4cb9-836b-011ee0425007, username=Johann-27.4)

In [6]:
val reportView = client.createReportView(user, reportViewName, fundName, client.fromYaml(File(REPORT_DATA_CONFIGURATION_YAML_FILE)))
reportView

ReportViewTO(id=8720be90-cb0a-46b5-b04f-46f6aae6e772, name=Expenses report, fundId=53ee50d0-87b5-4012-8baf-f8b4157e5269, dataConfiguration=ReportDataConfigurationTO(currency=Currency(value=RON), filter=RecordFilterTO(labels=[basic, home, shopping_services, transport, fun, gifts, development]), groups=null, features=ReportDataFeaturesConfigurationTO(net=NetReportFeatureTO(enabled=true, applyFilter=true), valueReport=GenericReportFeatureTO(enabled=true), groupedNet=GenericReportFeatureTO(enabled=false), groupedBudget=GroupedBudgetReportFeatureTO(enabled=false, distributions=[]), forecast=ForecastReportFeatureTO(inputBuckets=1))))

In [7]:
val monthlyReportData = client.getMonthlyReportViewData(user, reportViewName, monthlyReportFrom, monthlyReportTo, monthlyReportForecastUntil)


In [8]:
val yearlyData = client.getYearlyReportViewData(user, reportViewName, yearlyReportFrom, yearlyReportTo, yearlyReportForecastUntil)


In [9]:
import kotlinx.datetime.DateTimeUnit
import kotlinx.datetime.TimeZone
import kotlinx.datetime.atStartOfDayIn
import kotlinx.datetime.minus
import org.jetbrains.kotlinx.kandy.ir.Plot
import java.math.BigDecimal

fun plotGroupData(
    reportData: ReportDataTO,
    plotTitle: String,
    groupItemFilter: (ReportDataGroupedBudgetItemTO) -> Boolean,
): Plot {
    val dataFrame = dataFrameOf(
        "bucket" to reportData.data.map<ReportDataItemTO, LocalDate> { it.timeBucket.from },
        "spent" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.spent!! }.reduce { acc, value -> acc + value }
        },
        "allocated" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! }.reduce { acc, value -> acc + value }
        },
        "left" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.left!! }.reduce { acc, value -> acc + value }
        },
        "net" to reportData.data.map { bucket ->
            bucket.groupedBudget!!.filter(groupItemFilter).map { it.allocated!! + it.spent!! }
                .reduce { acc, value -> acc + value }
        },
    )
    return dataFrame
        .plot {
            line {
                val values =
                    listOf("spent", "allocated", "left").flatMap { dataFrame.get(it).values }.map { it as BigDecimal }
                val forecastBorderMin = minOf(BigDecimal.ZERO, values.minOrNull() ?: BigDecimal.ZERO)
                val forecastBorderMax = values.maxOrNull() ?: BigDecimal.ZERO
                val forecastBorderX = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> reportData.interval.toDate.minus(183, DateTimeUnit.DAY)
                    TimeGranularityTO.MONTHLY -> reportData.interval.toDate.minus(15, DateTimeUnit.DAY)
                    TimeGranularityTO.DAILY -> reportData.interval.toDate.minus(1, DateTimeUnit.DAY)
                }

                y(listOf(forecastBorderMin, forecastBorderMax))
                x.constant(forecastBorderX.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds())
            }
            x("bucket") {
                val format = when (reportData.interval.granularity) {
                    TimeGranularityTO.YEARLY -> "%Y"
                    TimeGranularityTO.MONTHLY -> "%b %Y"
                    TimeGranularityTO.DAILY -> "%d %b %Y"
                }
                axis.breaks(reportData.data.map {
                    it.timeBucket.from.atStartOfDayIn(TimeZone.UTC).toEpochMilliseconds()
                }, format)
            }
            line {
                y.constant(0)
            }
            line {
                y("spent")
                color = Color.RED
            }
            line {
                y("allocated")
                color = Color.GREEN
            }
            line {
                y("net")
                color = Color.YELLOW
            }
            area {
                y("left")
                borderLine {
                    color = Color.ORANGE
                }
            }
            layout {
                title = plotTitle
                size = 2400 to 1200
            }
        }
}

fun plotGroup(reportData: ReportDataTO, title: String, group: String): Plot =
    plotGroupData(reportData, title) { it.group == group }

fun plotGroupsTotal(reportData: ReportDataTO, title: String): Plot =
    plotGroupData(reportData, title) { true }



In [10]:
plotGroupsTotal(yearlyData, "Yearly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="kUIo6C"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Yearly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"left":[18160.866169342597,3544.0599255962934,2447.0380102675986,1350.0160949388967],
"spent":[-103911.4574768,-186186.29,-145048.8737384,-145048.8737384],
"net":[12394.376169342599,-14588.42,-1097.021915328701,-1097.021915328701],
"allocated":[116305.8336461426,171597.87,143951.8518230713,143951.8518230713]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5778368E12,1.6094592E12,1.6409952E12],
"format":"%Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"y":"y"
},
"stat":"identity",
"data":{
"y":[-186186.29,171597.87]
},
"sampling":"none",
"x":1.5935616E12,
"inherit_aes":false,
"position":"identity",
"geom":"line"
},{
"mapping":{
"x":"bucket"
},
"stat":"identity",
"sampling":"none",
"y":0.0,
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"spent"
},
"stat":"identity",
"color":"#ee6666",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"allocated"
},
"stat":"identity",
"color":"#3ba272",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"net"
},
"stat":"identity",
"color":"#fac858",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"bucket",
"y":"left"
},
"stat":"identity",
"color":"#fc8452",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"area",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"datetime",
"column":"bucket"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("kUIo6C");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2400.0,
 height: 1200.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2019 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2021 
 
 
 
 
 
 
 
 
 2022 
 
 
 
 
 
 
 
 
 
 
 -200,000 
 
 
 
 
 
 
 -180,000 
 
 
 
 
 
 
 -160,000 
 
 
 
 
 
 
 -140,000 
 
 
 
 
 
 
 -120,000 
 
 
 
 
 
 
 -100,000 
 
 
 
 
 
 

In [11]:
plotGroupsTotal(monthlyReportData, "Monthly total expenses")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="eg5mDA"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Monthly total expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"left":[8248.009999999998,10951.57,17067.75,17292.506022686404,16409.800981436005,17634.7274648476,14404.39275595681,-784.3493119067911,4894.2030591987095,9818.25348725471,11803.609861628207,18160.866169342607,24395.160795337106,29941.272837978107,27811.898532850508,29842.07818091991,23411.57458146261,12509.842806606006,-4729.743404802091,-2051.7871160347945,-56.0184978675995,-320.4356971755895,2719.6396873483145,3544.0599255963166,1941.838989054615,9125.704940841124,15143.624181962809,14090.46084862947,12780.193904185031,11896.37221437021,11847.6678837375,13231.549858885386,14507.507831962273],
"spent":[-4468.51,-4301.32,-9171.45,-6967.76,-9043.57,-7024.09,-11232.58,-25034.7199255,-4539.6390361,-6776.5,-9787.68,-5529.22,-4702.18,-5442.03,-35747.94,-17363.44,-18069.43,-20971.82,-27334.72,-15561.74,-8102.56,-10342.12,-8651.84,-13896.47,-15195.09,-6317.14,-7060.76,-14072.260833333334,-13797.995902777777,-13442.043061342592,-12814.561649787807,-11604.548453936792,-11274.78249176486],
"net":[2481.52,2703.56,6116.18,224.7560226864,-878.38,1231.8,-3232.75,-15187.9099255,5679.4044813615,4927.44,1994.64,6358.31,6231.96,5557.52,-2121.93,2032.08,-6427.61,-10905.23,-17239.74,2678.97,2008.06,-267.52,3042.74,822.28,-1599.09,7182.86,6034.24,-1053.1633333333336,-1310.2669444444446,-883.8216898148149,-48.7043306327155,1383.881975147891,1275.957973076881],
"allocated":[6950.03,7004.88,15287.63,7192.5160226864,8165.19,8255.89,7999.83,9846.81,10219.0435174615,11703.94,11782.32,11887.53,10934.14,10999.55,33626.01,19395.52,11641.82,10066.59,10094.98,18240.71,10110.62,10074.6,11694.58,14718.75,13596.0,13500.0,13095.0,13019.0975,12487.728958333333,12558.221371527778,12765.857319155093,12988.430429084683,12550.74046484174]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.551398

In [12]:
plotGroup(monthlyReportData, "Basic Expenses", "basic")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="H7G9ay"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Basic Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"left":[1509.9159999999997,1102.0699999999997,2340.4390000000003,2060.4334863521926,2237.9913498352944,2166.0131301380543,616.6538920664033,781.4769679456256,1953.5977074267869,2624.9997067648073,2575.2376446975654,3029.270256716548,3277.070477189104,3529.9290142313957,8601.913567218762,11726.135009319567,12217.940199433944,11179.629739373604,10157.423175114578,9818.022184116258,9505.70176910789,7357.861792049072,7874.232836583473,7639.759515639494,7839.020306331671,8707.491238533576,9232.208428996088,9319.412870662754,9153.48358246831,8932.648336924329,8780.088162585016,8700.009973717428,8641.72343577754],
"spent":[-2305.04,-2509.31,-3347.92,-2293.91,-2108.18,-2382.69,-3709.61,-2493.78,-1587.87,-2253.23,-2992.86,-2517.62,-2158.1,-2165.51,-2324.86,-948.25,-1951.84,-3154.12,-3041.34,-3989.72,-2393.26,-3636.25,-1883.73,-3184.42,-2513.19,-1835.59,-2078.71,-2550.868333333333,-2684.419861111111,-2745.46818287037,-2711.413864776234,-2683.920020174254,-2575.103355188775],
"net":[-220.031,-407.846,1238.369,-280.005513647808,178.0732,-71.0408,-1549.6559,164.8587,1171.271749714605,672.755,-47.28,454.2625,247.4108,254.391,5072.8622,3124.8092,492.9422,-1040.1361,-1022.344,-341.578,-371.136,-1621.33,455.186,-240.67,206.01,864.41,540.29,87.204441666667,-165.929288194444,-220.835245543981,-152.560174339313,-80.078188867589,-58.286537939888],
"allocated":[2085.009,2101.464,4586.289,2013.904486352192,2286.2532,2311.6492,2159.9541,2658.6387,2759.141749714605,2925.985,2945.58,2971.8825,2405.5108,2419.901,7397.7222,4073.0592,2444.7822,2113.9839,2018.996,3648.142,2022.124,2014.92,2338.916,2943.75,2719.2,2700.0,2619.0,2638.072775,2518.490572916667,2524.632937326389,2558.853690436921,2603.841831306665,2516.816817248887]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.554

In [13]:
plotGroup(monthlyReportData, "Home Expenses", "home")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="4cy0X2"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Home Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"left":[3314.956,4807.26,7629.649,9531.25412725965,11741.731058362084,13911.46764007915,16293.313485323306,857.7557199312646,4012.112809380651,8000.051345551148,12069.76851652298,16185.95246461429,20161.689210379238,21815.46095972219,5562.230976955584,-2622.245883268029,-10715.5354205046,-23660.724666779603,-42910.00393770309,-45970.17250574397,-45562.96980984446,-41258.56888948625,-40020.28769550255,-37269.15442277844,-41808.66965290709,-38371.47824878738,-34252.623233146165,-37732.39323314617,-40820.13599981283,-43490.72868036839,-45305.23840930357,-45666.87491565003,-45804.400147525346],
"spent":[-500.0,-609.16,-1763.9,-400.0,-400.0,-467.23,-420.0,-18881.0,-423.24,-456.78,-400.0,-400.0,-400.0,-2736.62,-29698.21,-16136.26,-12866.69,-17070.89,-23488.88,-10712.07,-3563.87,-2752.85,-3681.19,-3840.81,-10690.64,-2616.15,-1842.52,-9105.235,-8519.31625,-8157.035104166667,-7414.213862847222,-6074.658351417824,-5688.207380702643],
"net":[1585.009,1492.304,2822.389,1901.605127259648,2212.8608,2174.6548,2379.9405,-15434.6165,3153.425231111525,3990.7172,4077.2816,4117.2614,3973.656,1663.2,-16247.806,-8184.0968,-8093.5438,-12943.5881,-19248.9884,-3050.9718,682.5904,1780.72,1581.371,2782.6275,-4572.44,3458.85,4050.23,-3479.77,-3087.742766666667,-2670.592680555556,-1814.509728935185,-361.636506346451,-137.525231875322],
"allocated":[2085.009,2101.464,4586.289,2301.605127259648,2612.8608,2641.8848,2799.9405,3446.3835,3576.665231111525,4447.4972,4477.2816,4517.2614,4373.656,4399.82,13450.404,7952.1632,4773.1462,4127.3019,4239.8916,7661.0982,4246.4604,4533.57,5262.561,6623.4375,6118.2,6075.0,5892.75,5625.465,5431.573483333333,5486.442423611111,5599.704133912037,5713.021845071373,5550.682148827321]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688

In [14]:
plotGroup(monthlyReportData, "Transport Expenses", "transport")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ksdtOw"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Transport Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"left":[431.586,573.98,124.24149999999997,345.9676215880479,362.8043913747458,592.3847247539878,944.4122590704538,257.7825003352964,744.1160343937266,1675.5746829048828,349.89234735217747,-481.79588706437113,112.38757683679441,1161.0855208116468,4629.1679398680235,6956.314034664291,7133.757924451113,8140.004671179787,8863.952139571995,11237.144222938865,12174.411763022446,12505.508983816631,11340.821009646903,9464.840742949438,11190.98919817717,13040.746586961757,14568.504018237634,15439.9408599043,16190.042238376522,16987.80886505476,17768.290977289522,18553.49264887718,19206.520101430477],
"spent":[-204.24,-207.85,-1214.12,-281.75,-554.64,-348.18,-288.04,-1474.32,-331.47,-238.42,-2502.33,-2020.41,-608.51,-161.2,-230.49,0.0,-1218.9,-202.81,-588.5,0.0,-444.43,-405.02,-2905.92,-3945.53,-304.89,-181.04,-413.17,-884.1841666666667,-957.8661805555556,-936.1133622685185,-997.221975790895,-1031.282140440136,-1117.222318810148],
"net":[143.2615,142.394,-449.7385,221.726121588048,16.9233,229.7323,351.9464,-686.5752,486.05348139692,931.974,-1324.098,-831.657,594.2454,1048.7505,3468.3711,2327.4624,178.1184,1005.1808,723.8474,2371.2923,869.9506,1005.424,-1268.6788,-1884.905,1734.51,1843.96,1551.08,871.4368416666663,750.1013784722225,797.7666266782405,780.482112234761,785.201671587658,653.027452553295],
"allocated":[347.5015,350.244,764.3815,503.476121588048,571.5633,577.9123,639.9864,787.7448,817.52348139692,1170.394,1178.232,1188.753,1202.7554,1209.9505,3698.8611,2327.4624,1397.0184,1207.9908,1312.3474,2371.2923,1314.3806,1410.444,1637.2412,2060.625,2039.4,2025.0,1964.25,1755.621008333333,1707.967559027778,1733.879988946759,1777.704088025656,1816.483812027794,1770.249771363443]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593

In [15]:
plotGroup(monthlyReportData, "Shopping & Services Expenses", "shopping_services")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="7fWxEt"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Shopping & Services Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"left":[250.25599999999997,600.5,755.1115,-294.43753863881597,-741.4424966520149,-2740.7685311975124,-2644.33581289133,-1954.9063495023581,-1324.687096499022,-847.0638596533166,-21.500520487815592,810.6284988345645,1316.7911072102893,1976.14833357683,2733.4376033901626,3896.98203104808,4595.113365102493,5130.795380223344,5635.607455180325,6118.90081744503,6661.0875434491945,6695.121503265329,7141.578663842188,6842.828775902825,7307.58360084573,7041.379227161662,6258.316155910489,6575.761030910489,6822.682045493823,7031.970711292434,7214.117149240929,7369.3800403518,7497.407714055242],
"spent":[-385.57,0.0,-609.77,-1481.1,-936.99,-2494.99,-463.18,0.0,-82.99,-342.57,0.0,0.0,-149.99,0.0,-1259.78,0.0,0.0,-69.0,0.0,-429.95,0.0,0.0,-77.0,-893.12,-209.0,-944.99,-1425.22,-337.3566666666667,-365.4697222222222,-395.9255324074074,-423.1693267746914,-458.433437339249,-460.8070571175197],
"net":[-38.0685,350.244,154.6115,-1049.549038638816,-447.0786,-1999.6366,96.8081,689.2767,632.343046222305,476.7058,824.7624,832.1271,506.0584,659.973,757.7806,1163.7312,698.5092,534.9954,504.749,482.0855,505.531,402.984,390.7832,-304.37,470.8,-269.99,-770.47,317.444875,246.9210145833333,209.2886657986111,182.1464379484953,155.26289111087,128.0276737034425],
"allocated":[347.5015,350.244,764.3815,431.550961361184,489.9114,495.3534,559.9881,689.2767,715.333046222305,819.2758,824.7624,832.1271,656.0484,659.973,2017.5606,1163.7312,698.5092,603.9954,504.749,912.0355,505.531,402.984,467.7832,588.75,679.8,675.0,654.75,654.8015416666667,612.3907368055555,605.2141982060185,605.3157647231867,613.696328450119,588.8347308209622]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12

In [16]:
plotGroup(monthlyReportData, "Fun Expenses", "fun")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Y76odO"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Fun Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"left":[876.3179999999996,1357.0499999999995,2412.3045000000006,1238.3315829492326,63.73656293693919,1136.9755658232368,1147.0242885006066,150.8190993601525,-583.1817364387047,-51.962643010463985,890.672090139836,1741.5949810471618,1818.8572675554774,2477.97971172908,3207.5442110985587,4177.101043650251,4758.787016318047,5262.827588028949,5666.691486111626,6047.53441052026,5869.187410807603,6047.8298155854545,6682.850156611042,7424.04676927121,8097.288127542461,8776.482853008176,8712.536656235208,9192.119606235208,9630.853135401874,10057.640208665764,10478.04874636831,10899.84139554607,11325.147732155307],
"spent":[-1031.16,-570.0,-1237.89,-2109.0,-2235.76,0.0,-790.09,-1980.8199255,-1756.0690361,-405.5,0.0,-100.0,-579.01,0.0,-1287.38,0.0,0.0,0.0,0.0,-350.0,-619.0,0.0,0.0,0.0,0.0,0.0,-703.0,-139.3333333333333,-150.9444444444444,-163.5231481481481,-177.1500771604938,-191.912583590535,-178.7386322230796],
"net":[11.3445,480.732,1055.2545,-1173.972917050768,-1174.2853,1073.2657,9.893,-996.1389255,-734.16468435385,530.8152,942.5856,851.0024,77.0384,659.973,730.1806,969.776,582.091,503.3295,403.7992,379.6284,-214.5752,503.73,584.729,735.9375,679.8,675.0,-48.25,479.5829500000001,438.7335291666668,426.7870732638889,420.4085377025463,421.7926491777585,425.3063366092383],
"allocated":[1042.5045,1050.732,2293.1445,935.027082949232,1061.4747,1073.2657,799.983,984.681,1021.90435174615,936.3152,942.5856,951.0024,656.0484,659.973,2017.5606,969.776,582.091,503.3295,403.7992,729.6284,404.4248,503.73,584.729,735.9375,679.8,675.0,654.75,618.9162833333334,589.6779736111112,590.310221412037,597.5586148630401,613.7052327682935,604.0449688323179]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569

In [17]:
plotGroup(monthlyReportData, "Gifts Expenses", "gifts")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="c8jXn4"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Gifts Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"left":[1525.9824,1974.5680000000002,3661.2336,4122.335522722368,3171.127282388459,2830.505803270139,-1850.7852244216385,-972.3292044828909,-200.29074753497173,833.2025675715074,-1999.6664650830899,-1420.8002575627015,-1094.1389050853122,-323.87225074211074,2896.8892736854887,4637.963332897036,3796.695618634454,4328.921931867861,5122.4731490680015,6867.642032848577,6837.845148248435,6777.072705976606,7664.994540075155,8004.936557684583,7711.254739230997,8013.379483658733,8486.608050065279,8976.713750065279,9362.555591731945,9850.619086870833,10335.049623271296,10793.724537705131,11145.283111675119],
"spent":[0.0,-392.0,-147.85,-402.0,-1930.0,-1330.0,-5561.66,-204.8,-351.0,-20.0,-3892.49,-491.19,-766.57,-330.2,-141.92,-198.28,-2005.0,-475.0,-216.0,-80.0,-1082.0,-488.0,-104.0,-843.59,-1102.37,-511.87,-298.14,-617.0208333333333,-651.9159027777778,-539.1588946759259,-544.5054692322531,-571.8809250016075,-612.8710020850748],
"net":[834.0036,448.5856,1686.6656,461.101922722368,-950.1772,-339.2932,-4681.6787,878.3491,773.094786920765,1033.3546,-2832.0812,578.6877,326.844,769.755,3220.681,1741.272,-840.818,531.659,793.498,1744.071,-70.938,317.968,831.5664,333.91,-286.61,298.13,487.56,490.1056999999997,385.8418416666662,488.0634951388891,484.4305364004629,458.6749144338345,351.5585739699877],
"allocated":[834.0036,840.5856,1834.5156,863.101922722368,979.8228,990.7068,879.9813,1083.1491,1124.094786920765,1053.3546,1060.4088,1069.8777,1093.414,1099.955,3362.601,1939.552,1164.182,1006.659,1009.498,1824.071,1011.062,805.968,935.5664,1177.5,815.76,810.0,785.7,1107.126533333333,1037.757744444444,1027.222389814815,1028.936005632716,1030.555839435442,964.4295760550625]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12

In [18]:
plotGroup(monthlyReportData, "Development Expenses", "development")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="RjYmu6"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Development Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"left":[338.99559999999997,536.142,144.77090000000004,288.621220453728,-426.14716680950426,-261.8508680194531,-101.89013169099127,95.05195450611879,292.53608847024327,-2416.5483128738556,-2060.793751513447,-1703.9838872428859,-1197.4959387484841,-695.4584513509252,180.7149606339273,1069.8286126087132,1624.8158780271579,2128.3881627120636,2734.1131278544735,3829.1417218401857,4458.717677341292,1554.738391617568,2035.4501760921037,1436.8019869272068,1604.372669833676,1917.7038003045996,2138.0741056642764,2318.90596399761,2440.7133105253874,2526.41368593048,2577.311634285997,2581.976178337807,2495.8258843939348],
"spent":[-42.5,-13.0,-850.0,0.0,-878.0,-1.0,0.0,0.0,-7.0,-3060.0,0.0,0.0,-40.0,-48.5,-805.3,-80.65,-27.0,0.0,0.0,0.0,0.0,-3060.0,0.0,-1189.0,-375.0,-227.5,-300.0,-438.2625,-468.0635416666667,-504.8188368055556,-546.8870732060185,-592.4609959731868,-641.832745637619],
"net":[166.0009,197.1464,-391.3711,143.850320453728,-714.6962,164.1178,159.9966,196.9362,197.38087034923,-2708.8818,353.4696,356.6259,506.707,501.4775,876.0005,889.126,555.091,503.3295,605.6988,1094.4426,606.6372,-2657.016,467.7832,-600.25,168.84,312.5,223.8,180.8318583333333,121.8073465277777,85.7003754050925,50.897948355517,4.66454405181,-86.1502939438725],
"allocated":[208.5009,210.1464,458.6289,143.850320453728,163.3038,165.1178,159.9966,196.9362,204.38087034923,351.1182,353.4696,356.6259,546.707,549.9775,1681.3005,969.776,582.091,503.3295,605.6988,1094.4426,606.6372,402.984,467.7832,588.75,543.84,540.0,523.8,619.0943583333333,589.8708881944444,590.5192122106481,597.7850215615355,597.1255400249968,555.6824516937465]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.575158

In [19]:
plotGroup(monthlyReportData, "Investment Expenses", "investment")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="gIuhEv"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"ggtitle":{
"text":"Investment Expenses"
},
"mapping":{
},
"data":{
"bucket":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"left":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"spent":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"net":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
"allocated":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"ggsize":{
"width":2400.0,
"height":1200.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619392E12,1.5646176E12,1.567296E12,1.569888E12,1.5725664E12,1.5751584E12,1.5778368E12,1.5805152E12,1.5830208E12,1.5856992E12,1.5882912E12,1.5909696E12,1.5935616E12,1.59624E12,1.5989184E12,1.6015104E12,1.6041888E12,1.6067808E12,1.6094592E12,1.6121376E12,1.6145568E12,1.6172352E12,1.6198272E12,1.6225056E12,1.6250976E12,1.627776E12,1.6304544E12],
"format":"%b %Y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"datetime":true,
"breaks":[1.5463008E12,1.5489792E12,1.5513984E12,1.5540768E12,1.5566688E12,1.5593472E12,1.5619